<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Classes" data-toc-modified-id="Classes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Classes</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Environment</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main</a></span><ul class="toc-item"><li><span><a href="#First-Run" data-toc-modified-id="First-Run-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>First Run</a></span></li><li><span><a href="#Second-Run-using-Restart" data-toc-modified-id="Second-Run-using-Restart-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Second Run using Restart</a></span></li></ul></li><li><span><a href="#Scratch" data-toc-modified-id="Scratch-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scratch</a></span><ul class="toc-item"><li><span><a href="#Measure" data-toc-modified-id="Measure-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Measure</a></span></li></ul></li><li><span><a href="#Scratch" data-toc-modified-id="Scratch-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scratch</a></span></li></ul></div>

# Preamble

## Imports

In [3]:
import sys
sys.path.append('../')
sys.path.append('../model/')
from simfection_settings import SimfectionSettings
from model.simulation_run import SimulationRun

## Classes

## Functions

In [2]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

## Environment

# Main

## First Run

In [3]:
settings = {'num_days': 5}

In [4]:
sim = SimulationRun(settings)

+ Synthesizing population.


In [5]:
%%time 
sim.run()


+ Day 0
+ Population loaded.

+ Day 1
+ Population loaded.

+ Day 2
+ Population loaded.

+ Day 3
+ Population loaded.

+ Day 4
+ Population loaded.
CPU times: user 15.9 s, sys: 148 ms, total: 16.1 s
Wall time: 16.4 s


## Second Run using Restart

In [6]:
settings = {
    'previous_run': sim,
    'num_days': 2
}

In [7]:
sim2 = SimulationRun(settings)

+ Restarting from previous run.


In [8]:
%%time
sim2.run()


+ Day 5
+ Population loaded.

+ Day 6
+ Population loaded.
CPU times: user 7.61 s, sys: 60.9 ms, total: 7.67 s
Wall time: 7.88 s


# Scratch

In [ ]:
import sys
import numpy as np

## Measure

In [ ]:
get_size(sim)/1024/1024

np.array([get_size(day) for day in days]).sum()/1024/1024

days = sim.days

measurements = []

for i,day in enumerate(days):
    try:
        inf = day.starting_population.state.value_counts()['inf']
    except:
        inf = 0
    try:
        sus = day.starting_population.state.value_counts()['sus']
    except:
        sus = 0
    try:
        dead = day.starting_population.state.value_counts()['dead']
    except:
        dead = 0
        
    imm = sum(day.starting_population.immunity > 0)

    measure = {
        'day': i,
        'inf': inf,
        'sus': sus,
        'dead': dead,
        'imm': imm
    }

    measurements.append(measure)

data = pd.DataFrame(measurements)

data['sus_minus_imm'] = data.sus - data.imm

data['total_people'] = data.inf + data.sus

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
sns.set_context('notebook')

title = (
    '''
    No. People: {np}
    Avg. Connections: {ac}
    No. Days: {nd}
    Init. Infected: {ii}
    alpha: {ir}
    beta: {rr}
    
    '''
    .format(
        np=num_people,
        ac=mean_connections,
        ir=pathogen['infection_rate'],
        rr=pathogen['recovery_rate'],
        nd=num_days,
        ii=initial_states['inf']
    )
)
fig = plt.figure(figsize=(12,8))
ax = plt.gca()
sns.lineplot(x='day',y='sus_minus_imm',data=data,ax=ax,label='Susceptible')
sns.lineplot(x='day',y='inf',data=data,ax=ax,label='Infected')
sns.lineplot(x='day',y='imm',data=data,ax=ax,label='Recovered')
sns.lineplot(x='day',y='total_people',data=data,ax=ax,label='Total People')
ax.set(xlabel='Day',ylabel='Count',title=title)


infected_by = [pd.DataFrame(day.starting_population.infected_by.value_counts()) for day in days]

df = infected_by[1]
for day in infected_by[2:]:
    df += day
    

sns.set(context='notebook')
sns.barplot(x='index',y='infected_by',data=df.reset_index())

# Scratch

In [ ]:
from population_engine import PopulationEngine

In [ ]:
from simfection_settings import SimFectionSettings

In [ ]:
settings = SimFectionSettings(settings={
    'infection_rate': 0.9,
    'num_people': 5
})

In [ ]:
settings.get_setting('num_people')

In [ ]:
population = PopulationEngine(settings)

In [ ]:
population.synthesize_population()

In [ ]:
settings.get_setting('infection_rate')

In [ ]:
settings.set_setting('infection_rate',0.2)

In [ ]:
settings.settings